In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
cd drive/MyDrive/IE7374_Group14/colab/

/content/drive/MyDrive/IE7374_Group14/colab


In [3]:
pip install transformers datasets peft accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [4]:
import argparse
import logging
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)
from peft import LoraConfig, get_peft_model, TaskType

In [7]:
import pandas as pd

In [10]:
input_csv = pd.read_csv('v7_sample_train.csv')
input_csv.head(2)

,message_id,tag,anon_message
0,Message-ID: <30441432.1075842317918.JavaMail.e...,<|original|>,Will you be there tonight ? \n < ANON_NAME > \...
1,Message-ID: <19313841.1075852413904.JavaMail.e...,<|original|>,Hello ?


In [11]:
# data_prep_step1.py

import pandas as pd
import json
import re
import warnings

def extract_tag_index(tag: str) -> int:
    """
    Map message tags to an integer index for ordering:
      "<|original|>" → 0
      "<|replyN|>"   → N
    Anything else → large number (sorts last).
    """
    if tag == "<|original|>":
        return 0
    m = re.match(r"<\|reply(\d+)\|>", tag)
    if m:
        return int(m.group(1))
    return 9999

def prepare_pairs(input_csv: str, output_jsonl: str):
    # 1) Load the CSV, skipping malformed lines
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        df = pd.read_csv(
            input_csv,
            engine='python',
            on_bad_lines='skip'
        )

    # 2) Ensure there's a 'subject' column (blank for now)
    if 'subject' not in df.columns:
        df['subject'] = ""

    examples = []

    # 3) Process each thread
    for thread_id, group in df.groupby("message_id", sort=False):
        grp = group.copy()
        # 4) Sort messages by tag index
        grp['idx'] = grp['tag'].apply(extract_tag_index)
        grp = grp.sort_values('idx')

        msgs = grp['anon_message'].tolist()
        # 5a) Grab the subject from the original message row
        subject_rows = grp.loc[grp['idx'] == 0, 'subject']
        subject = subject_rows.iloc[0] if not subject_rows.empty else ""

        # 5b) For each reply position i (i >= 1), build one example
        for i in range(1, len(msgs)):
            examples.append({
                "thread": " ".join(msgs[:i]),  # all messages before reply i
                "subject": subject,            # blank or to-be-filled later
                "email": msgs[i-1],            # immediate predecessor
                "reply": msgs[i],              # this reply
                "tone": "[formal]"             # placeholder tone
            })

    # 6) Write examples to JSONL
    with open(output_jsonl, 'w', encoding='utf-8') as fout:
        for ex in examples:
            fout.write(json.dumps(ex, ensure_ascii=False) + "\n")

if __name__ == "__main__":
    input_csv    = "v7_sample_train.csv"
    output_jsonl = "sample_enron_pairs.jsonl"

    print(f"Reading {input_csv}…")
    prepare_pairs(input_csv, output_jsonl)
    count = sum(1 for _ in open(output_jsonl, encoding='utf-8'))
    print(f"Wrote {count} examples to {output_jsonl}.")


Reading v7_sample_train.csv…
Wrote 79 examples to sample_enron_pairs.jsonl.


In [13]:
# data_prep_step2.py

import json

def build_prompts(input_jsonl: str,
                  output_full: str,
                  output_subject: str):
    """
    Reads each example from input_jsonl (which has keys:
      'thread', 'subject', 'email', 'reply', 'tone')
    and writes two JSONL files, one per prompt format.
    """
    with open(input_jsonl, encoding='utf-8') as fin, \
         open(output_full,    'w', encoding='utf-8') as fout_full, \
         open(output_subject, 'w', encoding='utf-8') as fout_subj:

        for line in fin:
            ex = json.loads(line)

            # 1) Full-thread prompt
            prompt_full = (
                f"{ex['tone']} Thread: {ex['thread']} Reply: {ex['reply']}"
            )
            fout_full.write(
                json.dumps({"text": prompt_full}, ensure_ascii=False) + "\n"
            )

            # 2) Subject + last email prompt
            prompt_subj = (
                f"{ex['tone']} Subject: {ex['subject']} ┃ "
                f"Last message: {ex['email']} Reply: {ex['reply']}"
            )
            fout_subj.write(
                json.dumps({"text": prompt_subj}, ensure_ascii=False) + "\n"
            )

if __name__ == "__main__":
    input_jsonl    = "sample_enron_pairs.jsonl"
    output_full    = "sample_enron_prompts_full.jsonl"
    output_subject = "sample_enron_prompts_subject.jsonl"

    print("Building prompts…")
    build_prompts(input_jsonl, output_full, output_subject)
    print("Done.")


Building prompts…
Done.


In [14]:
# data_prep_step3.py

import os
from datasets import load_dataset
from transformers import AutoTokenizer

def tokenize_and_save(input_jsonl: str, output_dir: str, max_len: int = 512):
    """
    1) Loads the JSONL at `input_jsonl` (expects one {"text": ...} per line).
    2) Tokenizes to GPT-2 input_ids (truncates/pads to max_len).
    3) Sets labels = input_ids for causal LM.
    4) Saves the processed dataset to `output_dir` (creates it if needed).
    """
    # a) Load the prompts as a Hugging Face Dataset
    ds = load_dataset("json", data_files=input_jsonl, split="train")

    # b) Load GPT-2 tokenizer and set pad token
    tokenizer = AutoTokenizer.from_pretrained("gpt2")
    tokenizer.pad_token = tokenizer.eos_token

    # c) Tokenization function (batched)
    def tokenize_fn(batch):
        enc = tokenizer(
            batch["text"],
            truncation=True,
            max_length=max_len,
            padding="max_length"
        )
        # Use the inputs themselves as labels for next‐token prediction
        enc["labels"] = enc["input_ids"].copy()
        return enc

    # d) Apply it
    tokenized = ds.map(
        tokenize_fn,
        batched=True,
        remove_columns=["text"]
    )

    # e) Save to disk
    os.makedirs(output_dir, exist_ok=True)
    tokenized.save_to_disk(output_dir)
    print(f"> Saved {len(tokenized)} examples to '{output_dir}'")

if __name__ == "__main__":
    # Filepaths from Step 2
    full_in  = "sample_enron_prompts_full.jsonl"
    subj_in  = "sample_enron_prompts_subject.jsonl"

    # Output folders for Step 4’s training
    full_out = "tokenized_full"
    subj_out = "tokenized_subject"

    print("💬 Tokenizing full‐thread prompts …")
    tokenize_and_save(full_in, full_out)

    print("💬 Tokenizing subject+last‐email prompts …")
    tokenize_and_save(subj_in, subj_out)

    print("✅ Step 3 complete.")


💬 Tokenizing full‐thread prompts …


Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

...

> Saved 79 examples to 'tokenized_full'
💬 Tokenizing subject+last‐email prompts …


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/79 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/79 [00:00<?, ? examples/s]

> Saved 79 examples to 'tokenized_subject'
✅ Step 3 complete.


In [16]:
#!/usr/bin/env python
# train_step4_v2.py

import os
import logging
from datasets import load_from_disk
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)
from peft import LoraConfig, get_peft_model, TaskType

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def fine_tune(tokenized_dir, output_dir):
    # 1) Load and split
    ds = load_from_disk(tokenized_dir)
    ds = ds.train_test_split(test_size=0.1) if not isinstance(ds, dict) else ds
    train_ds = ds.get("train", ds.get("all"))
    eval_ds  = ds.get("test",  ds.get("validation"))

    logger.info(f"→ {tokenized_dir}: train={len(train_ds)}, eval={len(eval_ds)}")

    # 2) Model & tokenizer
    tokenizer = AutoTokenizer.from_pretrained("gpt2")
    tokenizer.pad_token = tokenizer.eos_token
    model = AutoModelForCausalLM.from_pretrained("gpt2")

    # 3) Inject LoRA
    lora_cfg = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=8, lora_alpha=32, lora_dropout=0.1,
    )
    model = get_peft_model(model, lora_cfg)

    # 4) Data collator
    data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

    # 5) Training args
    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=True,
        num_train_epochs=3,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        learning_rate=2e-4,
        eval_strategy="steps",
        eval_steps=500,
        save_steps=500,
        save_total_limit=2,
        logging_steps=100,
        report_to="none",
    )

    # 6) Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=eval_ds,
        data_collator=data_collator,
        tokenizer=tokenizer,
    )

    # 7) Train & save
    logger.info(f"✨ Fine‐tuning {tokenized_dir} → {output_dir}")
    trainer.train()
    trainer.save_model(output_dir)
    logger.info(f"✔ Done {output_dir}\n")

if __name__ == "__main__":
    pairs = [
        ("tokenized_full",    "outputs/gpt2_lora_full"),
        ("tokenized_subject", "outputs/gpt2_lora_subject"),
    ]
    for tok_dir, out_dir in pairs:
        os.makedirs(out_dir, exist_ok=True)
        fine_tune(tok_dir, out_dir)

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1803: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
/tmp/ipython-input-16-605580672.py:61: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1803: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
/tmp/ipython-input-16-605580672.py:61: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss,Validation Loss


In [17]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_dir = "outputs/gpt2_lora_full"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForCausalLM.from_pretrained(model_dir)

prompt = "Subject: Meeting follow-up\n\nHi team,"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generate output
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_length=100,
        do_sample=True,
        top_k=50,
        top_p=0.95
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Subject: Meeting follow-up

Hi team,

There have been several reports that we were seeing several more people attending our meeting, including the other members of our community, but were unable to get in touch with anyone.

We are sorry for that, but we have received some very personal messages from the members of your community which are absolutely, terribly regrettable and heart breaking. We are currently working to resolve this and will be contacting members and asking them to please feel free to


In [23]:
prompt = """Subject: Weekly Team Update

Hi everyone,
The client feedback for Project Atlas was very positive. Great work!
For those interested, we’ll do a quick walkthrough of the new CRM interface during Friday’s stand-up.
Thanks!
"""

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generate output
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_length=100,
        do_sample=True,
        top_k=50,
        top_p=0.95
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Subject: Weekly Team Update

Hi everyone,
The client feedback for Project Atlas was very positive. Great work!
For those interested, we’ll do a quick walkthrough of the new CRM interface during Friday’s stand-up.
Thanks!
Thank you for your continued support of Project Atlas and for the great community support. This morning you can be assured that we will be back next week with a new layout that will address some of the issues of the campaign
